In [1]:
import random
import time
import math
import numpy
import scipy
import networkx as nx
import matplotlib
import pylab
import matplotlib.pyplot as plt

## Louvain Method Implementation

In [3]:
class Community:
    """ Data structure to hold community information and calculate modularity """
    def __init__(self,name,intraCommWeights,allWeights,nodeDict):
        self.name = name
        self.intraCommWeights = intraCommWeights
        self.allWeights = allWeights
        self.nodeDict = nodeDict
    def getModularity(self,graphWeight,graphWeight_x2):
        self.modularity = (self.intraCommWeights/graphWeight) - \
                          ((self.allWeights/graphWeight_x2)**2)

class PreCompute:
    """ Data structure to hold network information.
        Drastically reduces compute time by holding information that would otherwise
        need to be looked up over and over again..."""
    def __init__(self,nDegrees,nNeighbs,nComms,nSelfW,edgeDict,tW,tW_x2):
        self.nDegrees = nDegrees # {nodeID:degreeOfNode}
        self.nNeighbs = nNeighbs # {nodeID:set(nodeNeighborIDs)}
        self.nComms = nComms     # {nodeID:communityID_nodeBelongsTo}
        self.nSelfW = nSelfW     # {nodeID:selfLoopWeightOfNode}
        self.edgeDict = edgeDict # {(node1,node2):weight} {(node2,node1):weight}
        self.tW = tW             # totalWeight of network
        self.tW_x2 = tW_x2       # (totalWeight of network) x 2

def randomizeOrder(rand,items):
    if rand == True:
        randomOrder = list(items)
        random.shuffle(randomOrder)
        return iter(randomOrder)
    else:
        return items

def initializeCommunities(graph,weight='weight'):
    ### communityDict = {communityName:communityObject}
    ### preComputeObj = PreCompute Object that stores information so we only have to compute it once
    preComputeObj = PreCompute({},{},{},{},{},0.0,0.0)
    communityDict = {}
    for node in graph.nodes():
        selfLoop = 0.0
        ### This could be replaced by a try statement. Would speed up for large networks ###
        if graph.has_edge(node,node):
            selfLoop = graph[node][node][weight]
        else:
            graph.add_edge(node,node,weight=selfLoop)
        nDegrees = graph.degree(node,weight=weight)
        preComputeObj.nDegrees.update({node:nDegrees})
        preComputeObj.nNeighbs.update({node:set(graph.neighbors(node))})
        preComputeObj.nComms.update({node:node})
        preComputeObj.nSelfW.update({node:selfLoop})
        communityDict.update({node:Community(node,selfLoop,nDegrees,{node:''})})
    
    for edge in graph.edges():
        edgeWeight = graph[edge[0]][edge[1]][weight]
        preComputeObj.edgeDict.update({edge:edgeWeight})
        preComputeObj.edgeDict.update({(edge[1],edge[0]):edgeWeight})
    
    preComputeObj.tW = graph.size(weight=weight)
    preComputeObj.tW_x2 = preComputeObj.tW * 2.0
    print "Initial communities found "+str(len(communityDict))
    return communityDict,preComputeObj

def initializeCommunities_priorComms(graph,levs,commKey='chrom',weight='weight'):
    ### communityDict = {communityName:communityObject}
    ### preComputeObj = PreCompute Object that stores information so we only have to compute it once
    preComputeObj = PreCompute({},{},{},{},{},0.0,0.0)
    communityDict = {}
    for node in graph.nodes():
        selfLoop = 0.0
        ### This could be replaced by a try statement. Would speed up for large networks ###
        if graph.has_edge(node,node):
            selfLoop = graph[node][node][weight]
        else:
            graph.add_edge(node,node,weight=selfLoop)
        nDegrees = graph.degree(node,weight=weight)
        priorComm = graph.node[node][commKey]
        if priorComm not in communityDict:
            communityDict.update({priorComm:Community(priorComm,selfLoop,nDegrees,{node:''})})
        else:
            comObj = communityDict[priorComm]
            comObj.intraCommWeights += selfLoop
            for comNode in comObj.nodeDict:
                if graph.has_edge(comNode,node):
                    comObj.intraCommWeights += graph[node][comNode][weight]
            
            comObj.allWeights += nDegrees
            comObj.nodeDict.update({node:''})

        preComputeObj.nDegrees.update({node:nDegrees})
        preComputeObj.nNeighbs.update({node:set(graph.neighbors(node))})
        preComputeObj.nComms.update({node:priorComm})
        preComputeObj.nSelfW.update({node:selfLoop})
    
    preComputeObj.tW = graph.size(weight=weight)
    preComputeObj.tW_x2 = preComputeObj.tW * 2.0
    nGraph = phaseTwo(graph,communityDict,preComputeObj,weight=weight)
    levs.append(preComputeObj.nComms.copy())
    communityDict,preComputeObj = initializeCommunities(nGraph,weight=weight)
    return nGraph,communityDict,preComputeObj,levs

def computeModularity(communities,pObj):
    mod = 0.0
    for comName,comObj in communities.items():
        if len(comObj.nodeDict) >= 1:
            comObj.getModularity(pObj.tW,pObj.tW_x2)
            mod += comObj.modularity
    ################################
    return mod

def calculateCost(comObj,node,nodeNeighbors,nodeSelfWeight,nodeDegree,eDict,w,w_x2,weight='weight'):
    intraEdges = [ eDict[(node,nei)] for nei in nodeNeighbors & set(comObj.nodeDict) - set([node]) ]
    intraEdgeWeights = float(sum(intraEdges))
    
    newIntraCommWeights = comObj.intraCommWeights - intraEdgeWeights - nodeSelfWeight
    newAllWeights = comObj.allWeights - nodeDegree
    newModScore = (newIntraCommWeights/w) - ((newAllWeights/w_x2)**2)

    return (newModScore-comObj.modularity),newIntraCommWeights,newAllWeights,newModScore

def calculateGain(comObj,node,nodeNeighbors,nodeSelfWeight,nodeDegree,eDict,w,w_x2,weight='weight'):
    newEdges = [ eDict[(node,nei)] for nei in nodeNeighbors & set(comObj.nodeDict) - set([node]) ]
    newEdgeWeights = float(sum(newEdges))

    newIntraCommWeights = comObj.intraCommWeights + newEdgeWeights + nodeSelfWeight
    newAllWeights = comObj.allWeights + nodeDegree
    newModScore = (newIntraCommWeights/w) - ((newAllWeights/w_x2)**2)
    
    return (newModScore-comObj.modularity),newIntraCommWeights,newAllWeights,newModScore
    
def phaseOne(graph,communityDict,pObj,random=False,weight='weight'):
    endLoopMinimum = .0000001
    modScores = []
    currentMod = computeModularity(communityDict,pObj)
    modScores.append(currentMod)
    ###########
    while True:
        endLoopMod,newMod = currentMod,currentMod
        count = 0
        for node in randomizeOrder(random,graph.nodes()):
            nodeDegree = pObj.nDegrees[node]
            neighbs = pObj.nNeighbs[node]
            nodeSelfLoop = pObj.nSelfW[node]
            comObj1 = communityDict[pObj.nComms[node]]
            bestNeighb,bestIncrease = '!',0.0
            bestIntraCommW,bestAllCommW = 0.0,0.0
            bestModScore = 0.0
            ##################################################################
            cost1,intraCommW1,allCommW1,comObj1NewMod = calculateCost(comObj1,
                                                        node,
                                                        neighbs,
                                                        nodeSelfLoop,
                                                        nodeDegree,
                                                        pObj.edgeDict,
                                                        pObj.tW,
                                                        pObj.tW_x2,
                                                        weight=weight)
            ##################################################################
            neighborsToCalc = { pObj.nComms[n]:communityDict[pObj.nComms[n]] \
                                for n in neighbs if pObj.nComms[n] != comObj1.name }
            ############################################################################
            for neighCommName,comObj2 in randomizeOrder(random,neighborsToCalc.items()):
                cost2,intraCommW2,allCommW2,comObj2NewMod = calculateGain(comObj2,
                                                            node,
                                                            neighbs,
                                                            nodeSelfLoop,
                                                            nodeDegree,
                                                            pObj.edgeDict,
                                                            pObj.tW,
                                                            pObj.tW_x2,
                                                            weight=weight)
                ########################
                increase = cost1 + cost2
                if increase > bestIncrease:
                    bestIncrease = increase
                    bestNeighb = neighCommName
                    bestComObj = comObj2
                    bestIntraCommW = intraCommW2
                    bestAllCommW = allCommW2
                    bestModScore = comObj2NewMod
                    
            #####################
            if bestNeighb != '!':
                bestComObj.intraCommWeights = bestIntraCommW
                bestComObj.allWeights = bestAllCommW
                bestComObj.nodeDict.update({node:''})
                bestComObj.modularity = bestModScore
                pObj.nComms[node] = bestComObj.name
                comObj1.intraCommWeights = intraCommW1
                comObj1.allWeights = allCommW1
                comObj1.modularity = comObj1NewMod
                del comObj1.nodeDict[node]
                currentMod += bestIncrease
            
            #count += 1
            #if count % 100 == 0:
            #    print "Nodes processed "+str(count)
        ##############################################
        if (currentMod - endLoopMod) < endLoopMinimum:
            killSignal = 0
            if len(modScores) == 1:
                killSignal = 1
            break
        else:
            print "Loop Completed"
            modScores.append(currentMod)
    ##############################################
    return communityDict,pObj,modScores,killSignal
               
def phaseTwo(graph,communityDict,pObj,weight='weight'):
    newGraph = nx.Graph()
    for comName,comObj in communityDict.items():
        if len(comObj.nodeDict) >= 1:
            neighborCommunities = {}
            for node in comObj.nodeDict:
                for neighb in pObj.nNeighbs[node]:
                    if neighb != node:
                        neighbCom = pObj.nComms[neighb]
                        if neighbCom not in neighborCommunities:
                            neighborCommunities.update({neighbCom:graph[node][neighb][weight]})
                        else:
                            neighborCommunities[neighbCom] += graph[node][neighb][weight]
            #####################################################
            for neighb,edgeWeight in neighborCommunities.items():
                newGraph.add_edge(comName,neighb,weight=edgeWeight)
            newGraph.add_edge(comName,comName,weight=comObj.intraCommWeights)
            
    newSize = round(newGraph.size(weight=weight),1)
    oldSize = round(pObj.tW,1)
    if newSize != oldSize:
        print "ERROR! Edge weights were lost somewhere... "
        print "OriginalSize "+str(pObj.tW)
        print "NewSize "+str(newSize)
    return newGraph
                        
def louvainModularityOptimization(graph,priorCommunities=False,random=False,commKey='chrom',weight='weight'):
    levels = []
    if priorCommunities == False:
        cDict,pObj = initializeCommunities(graph,weight=weight)
    else:
        newGraph,cDict,pObj,levels = initializeCommunities_priorComms(graph,levels,commKey=commKey,weight=weight)
    ############
    mScores = []
    roundNumber = 0
    while True:
        if (roundNumber == 0) and (priorCommunities == False):
            cDict,pObj,nmScores,killSig = phaseOne(graph,cDict,pObj,random,weight)
        else:
            cDict,pObj,nmScores,killSig = phaseOne(newGraph,cDict,pObj,random,weight)
        ####################################################
        print "Round"+str(roundNumber+1)+" phase1 completed"
        if killSig == 1:
            break
        mScores += nmScores
        ####################
        if (roundNumber == 0) and (priorCommunities == False):
            newGraph = phaseTwo(graph,cDict,pObj,weight)
        else:
            newGraph = phaseTwo(newGraph,cDict,pObj,weight)
        ####################################################
        print "Round"+str(roundNumber+1)+" phase2 completed"
        roundNumber += 1
        levels.append(pObj.nComms.copy())
        cDict,pObj = initializeCommunities(newGraph,weight)
        ###################################################
        ### This just prevents the algorithm from running to infinity ###
        if roundNumber > 100:
            print "ERROR... Rounds of algorithm has failed to converge after 100 rounds..."
            print "Either the graph is HUGE or there is a bug..."
            print "Exiting..."
            break
        
    return levels,mScores

def giveFinalCommunities(partitionLevels,writeToFile=False):
    """ Retrieves the final community assignment for all nodes in the graph.
        Loops through a list of dictionaries. The first dictionary is { originalNodeName:communityID }
        Then each successive dictionary, the communityID from the previous one is now the key, and the
        value is the new communityID found at that level of the algorithm."""
    nodeComms = {}
    for nodeName in partitionLevels[0]:
        i,comm = 1,partitionLevels[0][nodeName]
        while i < len(partitionLevels):
            comm = partitionLevels[i][comm]
            i += 1
        nodeComms.update({nodeName:comm})
    ########################
    if writeToFile != False:
        outFile = open(writeToFile,'w')
        outFile.write("#NodeID"+'\t'+"CommunityID")
        nodesToWrite = [ [node,comm] for node,comm in nodeComms.items() ]
        nodesToWrite = sorted(nodesToWrite,key=lambda x: x[0])
        for n in nodesToWrite:
            outFile.write(str(n[0])+'\t'+str(n[1])+'\n')
        outFile.close()
    ################
    return nodeComms

## Testing

In [4]:
### Returns a "caveMan" graph of x cliques of size y each
x,y = 500,50
testGraph = nx.connected_caveman_graph(x, y)
### We must add edgeWeights to the graph because that is required by my implementation
for edge in testGraph.edges():
    testGraph.add_edge(edge[0],edge[1],weight=1.0)
### The community partition should give x communities in the end if it's correct

In [9]:
startTime = time.time()
partitionLevels,mScores = louvainModularityOptimization(testGraph,random=False)
comms = giveFinalCommunities(partitionLevels)
stopTime = time.time()
print "RunTime = "+str(stopTime-startTime)+" seconds"

Initial communities found 25000
Loop Completed
Loop Completed
Round1 phase1 completed
Round1 phase2 completed
Initial communities found 500
Round2 phase1 completed
RunTime = 18.1270489693
